In [8]:
import re
from anthropic import Anthropic
from getpass import getpass

api_key = getpass("Enter your Claude API key: ")
client = Anthropic(api_key=api_key)

def get_article(topic):
    import wikipedia
    try:
        return wikipedia.summary(topic, auto_suggest=False)
    except Exception as e:
        return f"Could not find article for '{topic}': {e}"

article_search_tool = {
    "name": "get_article",
    "description": "Get a recent Wikipedia article about a person, place, or event.",
    "input_schema": {
        "type": "object",
        "properties": {
            "search_term": {"type": "string", "description": "The topic to search for."}
        },
        "required": ["search_term"]
    }
}

system_prompt = """
You will be asked a question by the user. 
If answering the question requires data you were not trained on, you can use the get_article tool to get the contents of a recent Wikipedia article about the topic. 
If you can answer the question without needing to get more information, please do so. 
Only call the tool when needed. 
"""

def extract_answer(text):
    """Extracts the content inside <answer>...</answer> tags."""
    match = re.search(r"<answer>(.*?)</answer>", text, re.DOTALL)
    return match.group(1).strip() if match else text.strip()

def answer_question_loop():
    while True:
        question = input("\nAsk a question (or type 'quit' to exit): ")
        if question.lower() in ["quit", "exit"]:
            print("Goodbye!")
            break

        prompt = f"""
        Answer the following question <question>{question}</question>
        When you can answer the question, keep your answer as short as possible and enclose it in <answer> tags.
        """

        messages = [{"role": "user", "content": prompt}]
        while True:
            response = client.messages.create(
                model="claude-3-sonnet-20240229",
                system=system_prompt,
                messages=messages,
                tools=[article_search_tool],
                max_tokens=1000
            )

            stop_reason = response.stop_reason
            content = response.content[-1]

            if stop_reason == "tool_use":
                tool_use = content
                search_term = tool_use.input["search_term"]
                print(f"Claude wants to get an article for: {search_term}")
                wiki_content = get_article(search_term)

                tool_result = {
                    "role": "user",
                    "content": [{
                        "type": "tool_result",
                        "tool_use_id": tool_use.id,
                        "content": wiki_content
                    }]
                }
                messages.append({"role": "assistant", "content": [tool_use]})
                messages.append(tool_result)
            else:
                final_answer = extract_answer(content.text)
                print(f"\nClaude's answer: {final_answer}")
                break


In [9]:

answer_question_loop()


/var/folders/0b/fc06lg051536pp20ltszsw_00000gp/T/ipykernel_67123/14582184.py:53: DeprecationWarning: The model 'claude-3-sonnet-20240229' is deprecated and will reach end-of-life on July 21st, 2025.
Please migrate to a newer model. Visit https://docs.anthropic.com/en/docs/resources/model-deprecations for more information.
  response = client.messages.create(


Claude wants to get an article for: Christopher Nolan awards and honors


/Users/anshika@browserstack.com/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/var/folders/0b/fc06lg051536pp20ltszsw_00000gp/T/ipykernel_67123/14582184.py:53: DeprecationWarning: The model 'claude-3-sonnet-20240229' is deprecated and will reach end-of-life on July 21st, 2025.
Please migrate to a newer model. Visit https://docs.anthropic.com/en/docs/resources/model-deprecations for more information.
  response = client.messages.create(


Claude wants to get an article for: Ben Stiller awards and honors
Claude wants to get an article for: Christopher Nolan
Claude wants to get an article for: Ben Stiller

Claude's answer: Christopher Nolan has more Oscars than Ben Stiller. Nolan has won 2 Academy Awards, while the information above does not mention Stiller winning any Oscars.
Goodbye!
